In [1]:
from google.colab import drive
drive.mount('/content/drive')
print("Drive mounted")

Mounted at /content/drive
Drive mounted


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Dynamic pricing /Cleaned_Retail_Store_Inventory.csv')
print("File loaded")

File loaded


In [3]:
df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality,Revenue,Discounted_Price
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn,4254.50,26.800
1,2022-01-02,S001,P0001,Groceries,West,116,81,104,92.94,27.95,10,Cloudy,0,30.89,Spring,2263.95,25.155
2,2022-01-03,S001,P0001,Electronics,West,154,5,189,5.36,62.70,20,Rainy,0,58.22,Winter,313.50,50.160
3,2022-01-04,S001,P0001,Groceries,South,85,58,193,52.87,77.88,15,Cloudy,1,75.99,Winter,4517.04,66.198
4,2022-01-05,S001,P0001,Groceries,South,238,147,37,150.27,28.46,20,Sunny,1,29.40,Winter,4183.62,22.768


In [4]:
df.columns

Index(['Date', 'Store ID', 'Product ID', 'Category', 'Region',
       'Inventory Level', 'Units Sold', 'Units Ordered', 'Demand Forecast',
       'Price', 'Discount', 'Weather Condition', 'Holiday/Promotion',
       'Competitor Pricing', 'Seasonality', 'Revenue', 'Discounted_Price'],
      dtype='object')

**Time-Based Feature Engineering**

---



**Input Columns Used**
1. date
2. seasonality
3. holiday/promotion

In [5]:
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
# Day, Month, Year
df['day'] = df['Date'].dt.day
df['month'] = df['Date'].dt.month
df['year'] = df['Date'].dt.year

# Day of Week (0 = Monday, 6 = Sunday)
df['day_of_week'] = df['Date'].dt.dayofweek

# Weekend Flag
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

In [7]:
df['season'] = df['Seasonality']

In [8]:
df['is_holiday'] = df['Holiday/Promotion']

In [9]:
df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,...,Seasonality,Revenue,Discounted_Price,day,month,year,day_of_week,is_weekend,season,is_holiday
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,...,Autumn,4254.50,26.800,1,1,2022,5,1,Autumn,0
1,2022-01-02,S001,P0001,Groceries,West,116,81,104,92.94,27.95,...,Spring,2263.95,25.155,2,1,2022,6,1,Spring,0
2,2022-01-03,S001,P0001,Electronics,West,154,5,189,5.36,62.70,...,Winter,313.50,50.160,3,1,2022,0,0,Winter,0
3,2022-01-04,S001,P0001,Groceries,South,85,58,193,52.87,77.88,...,Winter,4517.04,66.198,4,1,2022,1,0,Winter,1
4,2022-01-05,S001,P0001,Groceries,South,238,147,37,150.27,28.46,...,Winter,4183.62,22.768,5,1,2022,2,0,Winter,1


| Feature       | Description                    |
| ------------- | ------------------------------ |
| `day`         | Day of month                   |
| `month`       | Month number                   |
| `year`        | Year                           |
| `day_of_week` | Weekday index                  |
| `is_weekend`  | Weekend flag                   |
| `season`      | Seasonal indicator             |
| `is_holiday`  | Festival / promotion indicator |




---

Price-Based Feature

Columns Used

1. Date
2. Store ID
3. Product ID
4. Price
5. Discount
6. Discounted_Price

In [10]:
df = df.sort_values(by=['Store ID', 'Product ID', 'Date'])

In [11]:
df['price_lag_1'] = (
    df.groupby(['Store ID', 'Product ID'])['Price']
      .shift(1)
)

In [12]:
df['price_lag_7'] = (
    df.groupby(['Store ID', 'Product ID'])['Price']
      .shift(7)
)

In [13]:
df['price_change_pct'] = (
    (df['Price'] - df['price_lag_1']) / df['price_lag_1']
)

In [14]:
df['Discount']

,Discount
0,20
1,10
2,20
3,15
4,20
...,...
73095,5
73096,20
73097,10
73098,5


In [15]:
df['discount_pct'] = df['Discount'] / 100

In [16]:
df['price_after_discount'] = df['Price'] * (1 - df['discount_pct'])

In [17]:
# Lag Prices
df = df.sort_values(by=['Store ID', 'Product ID', 'Date'])

df['price_lag_1'] = (
    df.groupby(['Store ID', 'Product ID'])['Price']
      .shift(1)
)

df['price_lag_7'] = (
    df.groupby(['Store ID', 'Product ID'])['Price']
      .shift(7)
)

In [18]:
# Price Change Percentage
df['price_change_pct'] = (
    (df['Price'] - df['price_lag_1']) / df['price_lag_1']
)



---

Demand Features (Lag & Rolling)

In [19]:
# Sorting data
df = df.sort_values(by=['Store ID', 'Product ID', 'Date'])

In [20]:
# Lagged demand captures short-term and long-term memory effects.
df['sales_lag_1'] = (
    df.groupby(['Store ID', 'Product ID'])['Units Sold']
      .shift(1)
)

df['sales_lag_7'] = (
    df.groupby(['Store ID', 'Product ID'])['Units Sold']
      .shift(7)
)

df['sales_lag_30'] = (
    df.groupby(['Store ID', 'Product ID'])['Units Sold']
      .shift(30)
)

In [21]:
# Rolling Average Demand .
# Rolling averages smooth noise and show demand trend strength.
df['sales_roll_avg_7'] = (
    df.groupby(['Store ID', 'Product ID'])['Units Sold']
      .rolling(window=7, min_periods=1)
      .mean()
      .reset_index(level=[0,1], drop=True)
)

df['sales_roll_avg_30'] = (
    df.groupby(['Store ID', 'Product ID'])['Units Sold']
      .rolling(window=30, min_periods=1)
      .mean()
      .reset_index(level=[0,1], drop=True)
)

In [22]:
# Demand Volatility (Rolling Std Dev)
# Volatility shows demand stability vs unpredictability.
df['sales_volatility_7'] = (
    df.groupby(['Store ID', 'Product ID'])['Units Sold']
      .rolling(window=7, min_periods=1)
      .std()
      .reset_index(level=[0,1], drop=True)
)

df['sales_volatility_30'] = (
    df.groupby(['Store ID', 'Product ID'])['Units Sold']
      .rolling(window=30, min_periods=1)
      .std()
      .reset_index(level=[0,1], drop=True)
)

In [23]:
lag_cols = [
    'sales_lag_1', 'sales_lag_7', 'sales_lag_30',
    'sales_volatility_7', 'sales_volatility_30'
]

df[lag_cols] = df[lag_cols].fillna(0)



---
Price Elasticity

Price Elasticity of Demand (PED) measures how sensitive demand is to price changes.

Elasticity =
% Change in Demand / % Change in Price

*   High elasticity → customers are price-sensitive
*   Low elasticity → demand is stable even if price changes

Columns Used : Units Sold , Price

In [24]:
# % Change Features
df = df.sort_values(by=['Store ID', 'Product ID', 'Date'])

df['price_pct_change'] = (
    df.groupby(['Store ID', 'Product ID'])['Price']
      .pct_change()
)

df['demand_pct_change'] = (
    df.groupby(['Store ID', 'Product ID'])['Units Sold']
      .pct_change()
)

In [25]:
# Raw Elasticity
df['price_elasticity'] = (
    df['demand_pct_change'] / df['price_pct_change']
)

In [26]:
import numpy as np
df['price_elasticity'] = df['price_elasticity'].replace([np.inf, -np.inf], np.nan)
df['price_elasticity'] = df['price_elasticity'].fillna(0)

In [27]:
# Smooth Elasticity
df['elasticity_7d'] = (
    df.groupby(['Store ID', 'Product ID'])['price_elasticity']
      .rolling(window=7, min_periods=1)
      .mean()
      .reset_index(level=[0,1], drop=True) )

In [28]:
# Aggregate Elasticity at Product Level
product_elasticity = (
    df.groupby('Product ID')['elasticity_7d']
      .mean()
      .reset_index()
      .rename(columns={'elasticity_7d': 'avg_price_elasticity'}))

In [29]:
def elasticity_class(e):
    if abs(e) >= 1.5:
        return 'High'
    elif abs(e) >= 0.5:
        return 'Medium'
    else:
        return 'Low'

product_elasticity['elasticity_class'] = (
    product_elasticity['avg_price_elasticity']
    .apply(elasticity_class)
)

In [30]:
df = df.merge(product_elasticity, on='Product ID', how='left')



---

Inventory Feature Engineering

Columns Used : Inventory Level, Units Sold , Date, Product ID, Store ID

Shows how much inventory is available relative to recent demand.

Inventory Ratio = Inventory Level / Rolling Avg Demand

In [31]:
df['avg_demand_7d'] = (
    df.groupby(['Store ID', 'Product ID'])['Units Sold']
      .rolling(window=7, min_periods=1)
      .mean()
      .reset_index(level=[0,1], drop=True)
)

df['inventory_ratio'] = df['Inventory Level'] / (df['avg_demand_7d'] + 1)

High ratio → Overstock risk

Low ratio → Stock-out risk

Estimated number of days inventory will last at current demand.

Days Until Stock-Out = Inventory Level / Avg Daily Sales

In [32]:
df['days_until_stockout'] = df['Inventory Level'] / (df['avg_demand_7d'] + 1)

Low stock if inventory < 3 days of demand

In [33]:
df['low_stock_flag'] = (df['days_until_stockout'] < 3).astype(int)

Overstock if inventory > 15 days of demand.

In [34]:
df['overstock_flag'] = (df['days_until_stockout'] > 15).astype(int)



---

Profit Feature


In [36]:
#Profit per Unit
df['estimated_cost'] = df['Competitor Pricing']
df['profit_per_unit'] = df['Price'] - df['estimated_cost']

Profit Margin = Price / Profit per Unit​

In [37]:
df['profit_margin'] = df['profit_per_unit'] / df['Price']

In [38]:
df['daily_profit'] = df['profit_per_unit'] * df['Units Sold']



---

Interaction Features

In [39]:
df['weekend_price_interaction'] = df['is_weekend'] * df['Price']

In [40]:
season_map = {
    'Winter': 0,
    'Spring': 1,
    'Summer': 2,
    'Autumn': 3
}

df['season_encoded'] = df['season'].map(season_map)
df['season_discount_interaction'] = df['season_encoded'] * df['discount_pct']

In [41]:
df['inventory_price_interaction'] = df['Inventory Level'] * df['Price']

In [42]:
df['lowstock_price_interaction'] = df['low_stock_flag'] * df['Price']



---

Categorical Encoding

One-Hot Encoding (Low Cardinality)

Using for:

Category, Region, Weather Condition , season , elasticity_class

In [43]:
df = pd.get_dummies( df, columns=['Category', 'Region', 'Weather Condition', 'season', 'elasticity_class'] , drop_first=True )

Frequency Encoding (High Cardinality IDs)

For: Product ID , Store ID

In [44]:
product_freq = df['Product ID'].value_counts()
store_freq = df['Store ID'].value_counts()

df['product_freq_enc'] = df['Product ID'].map(product_freq)
df['store_freq_enc'] = df['Store ID'].map(store_freq)

In [45]:
df.drop(['Product ID', 'Store ID'], axis=1, inplace=True)

In [48]:
df.isnull().sum().sort_values(ascending=False).head()

,0
price_lag_7,700
price_pct_change,100
demand_pct_change,100
price_lag_1,100
price_change_pct,100


In [49]:
df.shape

(73100, 65)



---

Final Cleaning

In [52]:
df.isnull().sum().sort_values(ascending=False).head()

,0
price_lag_7,700
price_pct_change,100
demand_pct_change,100
price_lag_1,100
price_change_pct,100


In [53]:
df.fillna(0, inplace=True)

In [54]:
print("Duplicates before:", df.duplicated().sum())

Duplicates before: 0


In [55]:
outlier_cols = [
    'Price', 'Units Sold', 'Revenue',
    'inventory_ratio', 'days_until_stockout',
    'profit_per_unit', 'profit_margin'
]

In [56]:
for col in outlier_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    df[col] = df[col].clip(lower, upper)

In [62]:
df.isnull().sum().sum()

np.int64(0)

In [64]:
df.duplicated().sum()

np.int64(0)

In [59]:
df.shape

(73100, 65)

In [65]:
df.to_csv("PriceOptima_FeatureEngineered_Final.csv", index=False)